In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
# !pip3 install malaya-gpu -U --no-deps

In [3]:
with open('filtered-dumping-wiki.txt') as fopen:
    data = list(filter(None, fopen.read().split('\n')))
    
data = [i for i in data if len(i) >= 2]

len(data)

2037249

In [4]:
data[0]

'Dirk Jan Klaas "Klaas-Jan" Huntelaar (lahir 12 Ogos 1983) merupakan pemain bola sepak Belanda yang bermain di posisi penyerang.'

In [5]:
import malaya

model = malaya.dependency.transformer(model = 'xlnet', quantized = True)

In [6]:
tokenizer = malaya.preprocessing.TOKENIZER().tokenize

In [7]:
def reset_t(tagging):
    t = tagging.copy()
    for i in range(len(t)):
        t[i] = [t[i][0], 2]
    return t

def augment_3(t, graph, selected = ['compound', 'flat']):
    l = list(graph.nodes.items())
    for no, n in enumerate(l[1:]):
        n = n[1]
        if n['rel'] in selected and n['address'] - 1 == n['head']:
            c = t[n['head'] - 1].copy()
            c[1] = 3
            t[n['head'] - 1] = [t[n['address'] - 1][0], 3]
            t[n['address'] - 1] = c
        
    return t

In [8]:
d_object, tagging, indexing = model.predict(' '.join(tokenizer(data[0])))

In [9]:
graph = malaya.dependency.dependency_graph(tagging, indexing)

In [10]:
t = reset_t(tagging)

In [11]:
augment_3(t, graph)

[['Dirk', 2],
 ['Jan', 2],
 ['Klaas', 2],
 ['"', 2],
 ['Klaas-Jan', 2],
 ['"', 2],
 ['Huntelaar', 2],
 ['(', 2],
 ['lahir', 2],
 ['12', 2],
 ['Ogos', 2],
 ['1983', 2],
 [')', 2],
 ['merupakan', 2],
 ['bola', 3],
 ['sepak', 3],
 ['Belanda', 3],
 ['pemain', 3],
 ['yang', 2],
 ['bermain', 2],
 ['di', 2],
 ['posisi', 2],
 ['penyerang', 2],
 ['.', 2]]

In [ ]:
from tqdm import tqdm

results = []
for i in tqdm(range(len(data[:5000]))):
    try:
        d_object, tagging, indexing = model.predict(' '.join(tokenizer(data[i])))
        graph = malaya.dependency.dependency_graph(tagging, indexing)
        t = reset_t(tagging)
        results.append((t.copy(), augment_3(t, graph)))
    except Exception as e:
        print(e)
        pass

100%|█████████▉| 4992/5000 [07:39<00:00, 10.21it/s]

In [ ]:
import pickle

with open('dataset-tatabahasa.pkl', 'wb') as fopen:
    pickle.dump(results, fopen)